## Chatbot NLP pour le Système Hydraulique

Cette section implémente un assistant conversationnel pour interagir avec les modèles prédictifs et l'utilisateur.

In [ ]:
# Install necessary packages
!pip install pandas numpy scikit-learn joblib streamlit pyngrok nltk plotly

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from joblib import load, dump
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import streamlit as st
import plotly.graph_objects as go
import os
from google.colab import drive


# Download NLTK resources
nltk.download('punkt')
# Add this line to download the required resource for word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')

# Mount Google Drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define paths
base_path = '/content/drive/MyDrive/Projet_time_series/'
data_path = base_path + 'Data_csv/'
model_path = '/content/drive/MyDrive/Projet_time_series/Models/'
scaler_X_path = model_path + 'scaler_X_inverse.joblib'
scaler_y_path = model_path + 'scaler_y_inverse.joblib'
model_inverse_mlp_path = model_path + 'model_inverse_mlp.joblib'

# Load the inverse MLP model and scalers
try:
    model_inverse_mlp = load(model_inverse_mlp_path)
    scaler_X_inverse = load(scaler_X_path)
    scaler_y_inverse = load(scaler_y_path)
    print("Model and scalers loaded successfully.")
except Exception as e:
    print(f"Error loading model or scalers: {e}")

Model and scalers loaded successfully.


In [ ]:
# Load sensor data and profile
sensors = ['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'TS2', 'TS3', 'TS4', 'VS1', 'CE', 'CP', 'SE']
data_dict = {}

for sensor in sensors:
    df = pd.read_csv(f'{data_path}/{sensor}.csv')
    # Compute mean value per cycle
    data_dict[sensor] = df.mean(axis=1)

# Combine into a single DataFrame
data = pd.DataFrame(data_dict)
profile = pd.read_csv(f'{data_path}/profile.csv')

# Merge profile data
data = pd.concat([data, profile], axis=1)

# Display data shape and check for NaN
print(f"Data shape: {data.shape}")
print(f"NaN values: {data.isna().sum().sum()}")

Data shape: (2205, 21)
NaN values: 0


In [ ]:
class HydraulicNLPAssistant:
    def __init__(self, model, scaler_X, scaler_y):
        self.model = model
        self.scaler_X = scaler_X
        self.scaler_y = scaler_y
        self.stop_words = set(stopwords.words('english'))  # Adjust for French if needed
        self.intents = {
            'predict_se': ['predict', 'forecast', 'efficiency', 'se'],
            'optimize_se': ['optimize', 'best', 'settings', 'parameters', 'achieve'],
            'general': ['hello', 'help', 'info']
        }

    def preprocess_text(self, text):
        # Tokenize and clean text
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t not in self.stop_words and t.isalnum() or t.isdigit()]
        return tokens

    def detect_intent(self, tokens):
        # Detect intent based on keywords
        for token in tokens:
            for intent, keywords in self.intents.items():
                if token in keywords:
                    return intent
        return 'general'

    def extract_parameters(self, text):
        # Extract numerical values (e.g., target SE)
        numbers = re.findall(r'[-+]?[0-9]*\.?[0-9]+', text)
        return [float(num) for num in numbers]

    def predict_sensor_values(self, target_se):
        # Predict sensor values for a target SE
        try:
            se_scaled = self.scaler_X.transform(np.array([[target_se]]))
            sensors_scaled = self.model.predict(se_scaled)
            sensors_original = self.scaler_y.inverse_transform(sensors_scaled)
            return sensors_original[0]
        except Exception as e:
            return f"Error in prediction: {e}"

    def process_message(self, message):
        tokens = self.preprocess_text(message)
        intent = self.detect_intent(tokens)
        params = self.extract_parameters(message)

        if intent == 'predict_se':
            return {
                'type': 'prediction',
                'message': 'Prediction of SE is not implemented in this assistant. Please use the Streamlit app for SE predictions.'
            }
        elif intent == 'optimize_se':
            if params:
                target_se = params[0]
                if 0 <= target_se <= 100:
                    sensor_values = self.predict_sensor_values(target_se)
                    if isinstance(sensor_values, str):
                        return {
                            'type': 'error',
                            'message': sensor_values
                        }
                    sensor_dict = {col: round(val, 2) for col, val in zip(['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE'], sensor_values)}
                    return {
                        'type': 'optimization',
                        'message': f"To achieve SE of {target_se}%, set the following sensor values:",
                        'sensor_values': sensor_dict
                    }
                else:
                    return {
                        'type': 'error',
                        'message': 'Target SE must be between 0 and 100.'
                    }
            else:
                return {
                    'type': 'error',
                    'message': 'Please provide a target SE value (e.g., "Optimize for SE 95").'
                }
        else:
            return {
                'type': 'general',
                'message': 'Hello! I can help predict sensor settings to achieve a target efficiency (SE). Try saying: "Optimize for SE 95".'
            }

In [ ]:
# Initialize and test the NLP assistant
assistant = HydraulicNLPAssistant(model_inverse_mlp, scaler_X_inverse, scaler_y_inverse)

# Test queries
test_queries = [
    "Optimize for SE 95",
    "Predict efficiency",
    "Hello, what can you do?",
    "Achieve SE 110"
]

for query in test_queries:
    response = assistant.process_message(query)
    print(f"\nQuery: {query}")
    print(f"Response: {response}")


Query: Optimize for SE 95
Response: {'type': 'optimization', 'message': 'To achieve SE of 95.0%, set the following sensor values:', 'sensor_values': {'PS1': np.float64(95.13), 'PS2': np.float64(60.52), 'PS3': np.float64(-0.67), 'PS5': np.float64(-0.12), 'PS6': np.float64(0.25), 'EPS1': np.float64(572.61), 'FS1': np.float64(7.36), 'FS2': np.float64(3.16), 'TS1': np.float64(245.41), 'CE': np.float64(-299.56)}}

Query: Predict efficiency
Response: {'type': 'prediction', 'message': 'Prediction of SE is not implemented in this assistant. Please use the Streamlit app for SE predictions.'}

Query: Hello, what can you do?
Response: {'type': 'general', 'message': 'Hello! I can help predict sensor settings to achieve a target efficiency (SE). Try saying: "Optimize for SE 95".'}

Query: Achieve SE 110
Response: {'type': 'error', 'message': 'Target SE must be between 0 and 100.'}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
%%writefile app.py

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from joblib import load
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from tensorflow.keras.models import load_model

# Set page config as the FIRST Streamlit command
st.set_page_config(page_title="Hydraulic System Assistant", layout="wide")

# Download NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

# Custom CSS
st.markdown("""
    <style>
    .main-header { font-size: 36px; color: #1f77b4; }
    .sidebar .sidebar-content { background-color: #f0f2f6; }
    </style>
""", unsafe_allow_html=True)

# Load models and scalers
@st.cache_resource
def load_resources():
    try:
        # LSTM and GRU for SE prediction
        model_lstm_path = '/content/drive/MyDrive/Projet_time_series/Models/model_lstm_se.h5'
        model_gru_path = '/content/drive/MyDrive/Projet_time_series/Models/model_gru_se.h5'
        scaler_X_se_path = '/content/drive/MyDrive/Projet_time_series/Models/scaler_X_se.joblib'
        scaler_y_se_path = '/content/drive/MyDrive/Projet_time_series/Models/scaler_y_se.joblib'

        # MLP for inverse prediction
        model_mlp_path = '/content/drive/MyDrive/Projet_time_series/Models/model_inverse_mlp.joblib'
        scaler_X_mlp_path = '/content/drive/MyDrive/Projet_time_series/Models/scaler_X_inverse.joblib'
        scaler_y_mlp_path = '/content/drive/MyDrive/Projet_time_series/Models/scaler_y_inverse.joblib'

        model_lstm = load_model(model_lstm_path)
        model_gru = load_model(model_gru_path)
        scaler_X_se = load(scaler_X_se_path)
        scaler_y_se = load(scaler_y_se_path)
        model_mlp = load(model_mlp_path)
        scaler_X_mlp = load(scaler_X_mlp_path)
        scaler_y_mlp = load(scaler_y_mlp_path)

        return model_lstm, model_gru, scaler_X_se, scaler_y_se, model_mlp, scaler_X_mlp, scaler_y_mlp, None
    except Exception as e:
        return None, None, None, None, None, None, None, str(e)

model_lstm, model_gru, scaler_X_se, scaler_y_se, model_mlp, scaler_X_mlp, scaler_y_mlp, load_error = load_resources()

if load_error:
    st.error(f"Error loading resources: {load_error}")
    st.stop()

# Load sample data
@st.cache_data
def load_data():
    sensors = ['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE', 'SE']
    data_path = '/content/drive/MyDrive/Projet_time_series/Data_csv/'
    data_dict = {}
    for sensor in sensors:
        try:
            df = pd.read_csv(f'{data_path}/{sensor}.csv')
            data_dict[sensor] = df.mean(axis=1)
        except FileNotFoundError:
            st.warning(f"Data file for {sensor} not found. Skipping.")
            data_dict[sensor] = pd.Series(np.nan, index=range(2205))
    return pd.DataFrame(data_dict)

data = load_data()

# Predict SE for all cycles
@st.cache_data
def predict_se_all_cycles(model, data):
    sensor_columns = ['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE']
    X = data[sensor_columns].values
    X_scaled = scaler_X_se.transform(X)
    X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
    se_scaled = model.predict(X_reshaped, verbose=0)
    se_original = scaler_y_se.inverse_transform(se_scaled)
    return np.clip(se_original.flatten(), 0, 100)

se_pred_lstm_all = predict_se_all_cycles(model_lstm, data)
se_pred_gru_all = predict_se_all_cycles(model_gru, data)

# NLP Assistant Class
class HydraulicNLPAssistant:
    def __init__(self, model_lstm, model_gru, scaler_X_se, scaler_y_se, model_mlp, scaler_X_mlp, scaler_y_mlp):
        self.model_lstm = model_lstm
        self.model_gru = model_gru
        self.scaler_X_se = scaler_X_se
        self.scaler_y_se = scaler_y_se
        self.model_mlp = model_mlp
        self.scaler_X_mlp = scaler_X_mlp
        self.scaler_y_mlp = scaler_y_mlp
        self.stop_words = set(stopwords.words('french'))
        self.intents = {
            'predict_se': ['prédire', 'prévoir', 'efficacité', 'se'],
            'optimize_se': ['optimiser', 'meilleur', 'paramètres', 'atteindre'],
            'general': ['bonjour', 'aide', 'info']
        }
        self.sensor_columns = ['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE']

    def preprocess_text(self, text):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t not in self.stop_words and (t.isalnum() or t.isdigit())]
        return tokens

    def detect_intent(self, tokens):
        for token in tokens:
            for intent, keywords in self.intents.items():
                if token in keywords:
                    return intent
        return 'general'

    def extract_parameters(self, text):
        numbers = re.findall(r'[-+]?[0-9]*\.?[0-9]+', text)
        return [float(num) for num in numbers]

    def validate_sensors(self, sensor_values):
        warnings = []
        for col, val in zip(self.sensor_columns, sensor_values):
            if col.startswith('PS') and val < 0:
                warnings.append(f"Avertissement: {col} pression ({val:.2f}) est négative.")
            elif col.startswith('TS') and (val < 0 or val > 100):
                warnings.append(f"Avertissement: {col} température ({val:.2f}) hors plage 0-100°C.")
            elif col == 'CE' and (val < 0 or val > 100):
                warnings.append(f"Avertissement: {col} efficacité ({val:.2f}) hors plage 0-100%.")
        return warnings

    def predict_se(self, sensor_values, model):
        try:
            if len(sensor_values) != len(self.sensor_columns):
                return None, [f"Erreur: {len(sensor_values)} valeurs fournies, {len(self.sensor_columns)} attendues ({', '.join(self.sensor_columns)})."]
            sensor_df = pd.DataFrame([sensor_values], columns=self.sensor_columns)
            sensor_scaled = self.scaler_X_se.transform(sensor_df)
            sensor_reshaped = sensor_scaled.reshape((sensor_scaled.shape[0], 1, sensor_scaled.shape[1]))
            se_scaled = model.predict(sensor_reshaped, verbose=0)
            se_original = self.scaler_y_se.inverse_transform(se_scaled)
            se_clipped = np.clip(se_original[0][0], 0, 100)
            return se_clipped, []
        except Exception as e:
            return None, [f"Erreur de prédiction SE: {e}"]

    def predict_sensor_values(self, target_se):
        try:
            se_scaled = self.scaler_X_mlp.transform(np.array([[target_se]]))
            sensors_scaled = self.model_mlp.predict(se_scaled)
            sensors_original = self.scaler_y_mlp.inverse_transform(sensors_scaled)
            warnings = self.validate_sensors(sensors_original[0])
            return sensors_original[0], warnings
        except Exception as e:
            return np.zeros(len(self.sensor_columns)), [f"Erreur de prédiction inverse: {str(e)}"]

    def process_message(self, message):
        tokens = self.preprocess_text(message)
        intent = self.detect_intent(tokens)
        params = self.extract_parameters(message)

        if intent == 'predict_se':
            if len(params) == len(self.sensor_columns):
                sensor_values = params
                se_lstm, warnings_lstm = self.predict_se(sensor_values, self.model_lstm)
                se_gru, warnings_gru = self.predict_se(sensor_values, self.model_gru)
                if se_lstm is not None and se_gru is not None:
                    return {
                        'type': 'prediction',
                        'message': f"Efficacité système (SE) prédite:\n- LSTM: {se_lstm:.2f}%\n- GRU: {se_gru:.2f}%",
                        'warnings': warnings_lstm + warnings_gru
                    }
                else:
                    return {
                        'type': 'error',
                        'message': warnings_lstm[0] if warnings_lstm else warnings_gru[0]
                    }
            else:
                return {
                    'type': 'error',
                    'message': f"Veuillez fournir {len(self.sensor_columns)} valeurs de capteurs dans l'ordre: {', '.join(self.sensor_columns)}."
                }
        elif intent == 'optimize_se':
            if params:
                target_se = params[0]
                if 0 <= target_se <= 100:
                    sensor_values, warnings = self.predict_sensor_values(target_se)
                    if sensor_values is not None:
                        sensor_dict = {col: round(val, 2) for col, val in zip(self.sensor_columns, sensor_values)}
                        return {
                            'type': 'optimization',
                            'message': f"Pour atteindre SE de {target_se}%, définissez les valeurs suivantes:",
                            'sensor_values': sensor_dict,
                            'warnings': warnings
                        }
                    else:
                        return {
                            'type': 'error',
                            'message': warnings[0]
                        }
                else:
                    return {
                        'type': 'error',
                        'message': 'SE cible doit être entre 0 et 100.'
                    }
            else:
                return {
                    'type': 'error',
                    'message': 'Veuillez fournir une valeur SE cible (ex: "Optimiser pour SE 95").'
                }
        else:
            return {
                'type': 'general',
                'message': 'Bonjour! Je peux optimiser les paramètres pour une efficacité cible (SE) ou prédire SE à partir des capteurs. Ex: "Optimiser pour SE 95" ou "Prédire SE avec 95.13, 60.52, 0, 0, 0.25, 572.32, 7.34, 3.1, 40, 20".'
            }

# Initialize assistant
assistant = HydraulicNLPAssistant(model_lstm, model_gru, scaler_X_se, scaler_y_se, model_mlp, scaler_X_mlp, scaler_y_mlp)

# Main UI
st.markdown('<h1 class="main-header"> Hydraulic System Assistant</h1>', unsafe_allow_html=True)

# Sidebar navigation
st.sidebar.title("Navigation")
page = st.sidebar.selectbox("Choisir une page:", ["Tableau de Bord", "Chatbot", "Prédiction SE", "Prédiction Inverse"])

# Dashboard Page
if page == "Tableau de Bord":
    st.header(" Tableau de Bord")
    st.write("Vue d'ensemble de l'efficacité (SE) et des données des capteurs.")

    # Plot SE over cycles (Real vs Predicted LSTM and GRU)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['SE'], mode='lines', name='Efficacité Réelle (SE)'))
    fig.add_trace(go.Scatter(x=data.index, y=se_pred_lstm_all, mode='lines', name='Efficacité Prédite (LSTM)', line=dict(dash='dash')))
    fig.add_trace(go.Scatter(x=data.index, y=se_pred_gru_all, mode='lines', name='Efficacité Prédite (GRU)', line=dict(dash='dot')))
    fig.update_layout(title='Efficacité Réelle vs Prédite par Cycle', xaxis_title='Cycle', yaxis_title='SE (%)')
    st.plotly_chart(fig)

    # Correlation heatmap
    st.subheader("Corrélations des Capteurs")
    corr = data[['SE', 'PS1', 'FS1', 'TS1', 'CE']].corr()
    fig = go.Figure(data=go.Heatmap(z=corr.values, x=corr.columns, y=corr.columns, colorscale='Viridis'))
    st.plotly_chart(fig)

# Chatbot Page
elif page == "Chatbot":
    st.header("🤖 Assistant NLP")
    st.write("Interagissez avec l'assistant pour optimiser l'efficacité ou prédire SE.")

    user_input = st.text_input("Entrez votre message:", "")
    if st.button("Envoyer"):
        response = assistant.process_message(user_input)
        if response['type'] == 'prediction':
            st.success(response['message'])
            if response.get('warnings'):
                st.warning("\n".join(response['warnings']))
        elif response['type'] == 'optimization':
            st.success(response['message'])
            if response.get('sensor_values'):
                st.write("Valeurs des capteurs:", response['sensor_values'])
            if response.get('warnings'):
                st.warning("\n".join(response['warnings']))
        elif response['type'] == 'error':
            st.error(response['message'])
        else:
            st.info(response['message'])

# Prediction SE Page
elif page == "Prédiction SE":
    st.header(" Prédiction de l'Efficacité Système (SE)")
    st.write("Entrez les valeurs des capteurs pour prédire SE avec LSTM et GRU.")

    sensor_columns = ['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE']
    sensor_values = []
    for sensor in sensor_columns:
        value = st.number_input(f"Valeur pour {sensor}", value=0.0, step=0.1)
        sensor_values.append(value)

    if st.button("Prédire SE"):
        warnings = assistant.validate_sensors(sensor_values)
        if warnings:
            st.warning("\n".join(warnings))
        if len(sensor_values) == len(sensor_columns):
            input_df = pd.DataFrame([sensor_values], columns=sensor_columns)
            input_scaled = scaler_X_se.transform(input_df)
            input_reshaped = input_scaled.reshape((1, 1, input_scaled.shape[1]))

            se_lstm_scaled = model_lstm.predict(input_reshaped, verbose=0)
            se_lstm = scaler_y_se.inverse_transform(se_lstm_scaled)[0][0]
            se_gru_scaled = model_gru.predict(input_reshaped, verbose=0)
            se_gru = scaler_y_se.inverse_transform(se_gru_scaled)[0][0]

            st.success(f"Prédiction SE (LSTM): {se_lstm:.2f}%")
            st.success(f"Prédiction SE (GRU): {se_gru:.2f}%")

            fig = go.Figure()
            fig.add_trace(go.Bar(x=['LSTM', 'GRU'], y=[se_lstm, se_gru], name='Prédiction SE'))
            fig.update_layout(title='Comparaison des Prédictions SE', yaxis_title='SE (%)', yaxis_range=[0, 100])
            st.plotly_chart(fig)
        else:
            st.error(f"Veuillez fournir {len(sensor_columns)} valeurs de capteurs.")

# Prediction Inverse Page
elif page == "Prédiction Inverse":
    st.header("Prédiction Inverse (SE vers Capteurs)")
    st.write("Entrez une valeur cible pour SE pour prédire les valeurs des capteurs.")

    target_se = st.number_input("Cible SE (%)", min_value=0.0, max_value=100.0, value=50.0)
    if st.button("Prédire Capteurs"):
        se_scaled = scaler_X_mlp.transform(np.array([[target_se]]))
        sensors_scaled = model_mlp.predict(se_scaled)
        sensors_original = scaler_y_mlp.inverse_transform(sensors_scaled)[0]
        sensor_dict = {col: round(val, 2) for col, val in zip(['PS1', 'PS2', 'PS3', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'CE'], sensors_original)}

        warnings = []
        for col, val in sensor_dict.items():
            if col.startswith('PS') and val < 0:
                warnings.append(f"Avertissement: {col} pression ({val:.2f}) est négative.")
            elif col.startswith('TS') and (val < 0 or val > 100):
                warnings.append(f"Avertissement: {col} température ({val:.2f}) hors plage 0-100°C.")
            elif col == 'CE' and (val < 0 or val > 100):
                warnings.append(f"Avertissement: {col} efficacité ({val:.2f}) hors plage 0-100%.")

        st.success(f"Prédiction des valeurs des capteurs pour SE = {target_se}%:")
        st.write(sensor_dict)
        if warnings:
            st.warning("\n".join(warnings))

# Run the app
if __name__ == "__main__":
    st.sidebar.write("Dernière mise à jour: 07:47 PM +01, 08/06/2025")

Overwriting app.py


In [ ]:
# Obtenir l'adresse IP publique
!wget -q -O - ipv4.icanhazip.com

# Démarrer l'application Streamlit et exposer avec localtunnel
!streamlit run app.py & npx localtunnel --port 8501

35.227.50.79


⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.50.79:8501

⠴⠦your url is: https://three-bats-stay.loca.lt
2025-06-13 13:39:22.725261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749821962.757348   34971 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749821962.767468   34971 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 13:39:22.798178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable t